# Tutorial 4: Multi-level column indices (`MultiIndex`)

A MultiIndex is an index with multiple, hierarchical levels. We use a multiindex for the column headers of a dataframe. This allows each column to have multiple keys associated with it--one key for each level.

The multiindex is a native part of the `pandas` package. For more documentation, see their [MultiIndex / advanced indexing page](https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html).

MultiIndexes make it easy to associate multiple identifiers with a single column. In this tutorial, we'll work with phosphoproteomics data from the endometrial cancer dataset.

Phosphoproteomics is the large-scale study of proteins that have undergone a process called phosphorylation. This is particularly important in the context of disease, as the process is often disrupted in cancer cells.

Let's start by importing our required package and loading the endometrial dataset.

In [1]:
import cptac
en = cptac.Ucec()

Let's see the data sources available in our dataset.

In [2]:
en.list_data_sources()

Here, we see different types of omics data (e.g., CNV, proteomics, phosphoproteomics) available from various sources (e.g., washu, umich).

We will retrieve phosphoproteomics, proteomics and CNV data from the respective sources.

In [3]:
#Some data may take several minutes to load
en.get_proteomics('umich')
en.get_phosphoproteomics('umich')
en.get_CNV('washu')

Now that our data is loaded, let's take a look at the structure of the phosphoproteomics data.

In [4]:
# Display first 5 rows of phosphoproteomics data
en.get_phosphoproteomics('umich').head()

# Join functions with multiindices 
The join functions have been written to handle multiindices. More information on the join functions can be found in the joining_dataframes tutorial. 
An example of joining a multiindexed dataframe (in this case phosphoproteomics) with a non multiindexed dataframe (in this case CNV) is below. 

In [5]:
phospho_and_CNV = en.join_omics_to_omics(df1_name="CNV", df2_name="phosphoproteomics", df1_source='washu', df2_source = 'umich')
phospho_and_CNV.head()

Since the C3L-00084 row doesn't have CNV data, it is filled in with NANs, so that it can be joined to the CNV dataframe. 

# How to select from multiindex


## Selecting based on all levels
We can select single columns by passing the proper keys for all levels of the multiindex. For example, to get the proteomics for ARF5, we'd do the following:

In [6]:
prot = en.get_proteomics('umich')
all_levels_selection = prot["ARF5"]

#Display the first 10 rows of the desired data
all_levels_selection.head(10)

## Selecting based on one level
We can easily select multiple columns from our multiindex dataframe, based on just the "Name" level of the multiindex:

In [7]:
gene1_filter = prot.columns.get_level_values("Name").str.startswith("A") # Select all columns where the gene starts with "A". This will grab every column where the key "Name" starts with AA
gene1_data = prot.loc[:, gene1_filter]
gene1_data.head()

## Selecting based on a different level of the multiindex
We can also select based on one of the inner levels of the multiindex. For example, to get data for all tyrosine phosphorylation sites:

In [8]:
y_site_filter = prot.columns.get_level_values("Database_ID").str.contains("ENSp") # Create a boolean filter selecting all columns where the Site level contains a "Y"

y_sites = prot.loc[:, y_site_filter] # Select the columns
y_sites.head()

# How to use `cptac.utils.reduce_multiindex()`
To make it easier to work with multi-level indices, we provide the `reduce_multiindex` function, available for import from the `cptac.utils` submodule. It can both drop levels from a multiindex, and "flatten" a multi-level index into a single-level index by concatenating the keys from multiple levels into a single key for each column.

In [9]:
import cptac.utils as ut

## Dropping Levels
We can drop levels based on index or name. We can also drop single or multiple levels at once. 
Note that it will warn you if duplicate column key combinations arise due to dropping levels. 

### Dropping by index or name

In [12]:
ut.reduce_multiindex(df=prot, levels_to_drop="Database_ID").head()

### Dropping single or multiple levels at once
By passing a list (or array-like) to levels_to drop, we can drop multiple levels of the multiindex at the same time. Note that we must leave at least one existing level. 

We will show this with the colon data.

In [13]:
colon = cptac.Coad()
prot = colon.get_proteomics('umich')
prot.head()
#cptac.get_cancer_info()

In [14]:
# Drop level 'Name'
ut.reduce_multiindex(df=prot, levels_to_drop='Name').head()
#You can also pass a list in order to drop multiple levels

## Combining levels (Flattening)

We can combine levels of a multiindexed dataframe. When combined the levels will be sepereated by an underscore, by default. We could specify a different seperator using the `sep` parameter.

In [ ]:
ut.reduce_multiindex(df=prot, flatten=True).head()

When flatteing levels , NaNs and empty strings will automitically be dropped.

In [ ]:
phospho_and_CNV = en.join_omics_to_omics(df1_name="CNV", df2_name="phosphoproteomics", df1_source = 'washu', df2_source = 'umich')
phospho_and_CNV.head()

# Note that the CNV columns all have empty strings in the "Site" level of the columns,
# since the CNV data doesn't have any values for that.

In [ ]:
ut.reduce_multiindex(df=phospho_and_CNV, flatten=True).head()
# Notice that the empty strings have been dropped

## Getting a single level index of tuples

You can also use `reduce_multiindex` to turn the multi-level column index into a single level index of tuples, with each value in a column's tuple corresponding to the column's value for that level of the index:

In [ ]:
ut.reduce_multiindex(df=prot, tuples=True).head()

## Turning off warnings

If your multiindex operation creates duplicate column headers, or has no effect, `reduce_multiindex` will warn you. You can silence these warnings by passing `True` to the `quiet` parameter:

In [ ]:
ut.reduce_multiindex(df=prot, levels_to_drop="Name").head()

In [ ]:
# No warning will be issued
ut.reduce_multiindex(df=prot, levels_to_drop="Name", quiet=True).head()